## Stitching images

In [75]:
import cv2
import numpy as np
cv2.__version__

'4.1.1'

In [76]:
paths=[]
k=int(input("Enter number of images:"))
print("Enter image paths:")
for i in range(k):
    paths.append(input())
images=[]
for imagePath in paths:
    im=cv2.imread(imagePath,cv2.IMREAD_COLOR)
    #im=cv2.resize(im,(224,224))
    images.append(im)


Enter number of images:3
Enter image paths:
.\images1.jpg
.\images2.jpg
.\images3.jpg


## Displaying the images selected from given paths

In [77]:
for image in images:
    cv2.imshow('images',image)
    cv2.waitKey(0)
cv2.destroyAllWindows()

## Stitching images and displaying the stitched image

In [78]:
stitcher = cv2.Stitcher_create()
(status, stitched) = stitcher.stitch(images)
print(status)
"""cv2.imshow("Stitched", stitched)
cv2.waitKey(0)"""
if status == 0:
    # write the output stitched image to disk
    #cv2.imwrite(args["output"], stitched)
    # display the output stitched image to our screen
    cv2.imshow("Stitched_image", stitched)
    cv2.waitKey(0)

0


In [79]:
"""output_path="F:/Bublu/Internship/data/stitched_image.jpg"
cv2.imwrite(output_path,stitched)""" 

'output_path="F:/Bublu/Internship/data/stitched_image.jpg"\ncv2.imwrite(output_path,stitched)'

## Detecting people using YOLOv3

In [80]:
from __future__ import division
import time
import torch 
import torch.nn as nn
import torchvision.transforms as T
from torch.autograd import Variable
import numpy as np
import cv2
import import_ipynb 
from util import *
import argparse
import os 
import os.path as osp
from Yolov3 import Darknet
import pickle as pkl
import pandas as pd
import random
import glob
import matplotlib.pyplot as plt
from PIL import Image

In [81]:
CUDA=False
num_classes = 80    #For COCO
classes = load_classes(r".\YOLOv3_folder\coco_names.txt")   #loading classes

In [82]:
model = Darknet(r".\YOLOv3_folder\cfg\darknet\cfg\yolov3.cfg")  #loading model configuration
model.load_weights(r".\YOLOv3_folder\yolov3(1).weights")        # loading weights
print("Network successfully loaded")
nms_thesh=0.4
confidence=0.5
model.net_info["height"] = 416
inp_dim = int(model.net_info["height"])
assert inp_dim % 32 == 0 
assert inp_dim > 32

#If there's a GPU availible, put the model on GPU
if CUDA:
    model.cuda()


#Set the model in evaluation mode
model.eval()

Network successfully loaded


Darknet(
  (module_list): ModuleList(
    (0): Sequential(
      (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch_norm_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_0): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (1): Sequential(
      (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_1): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (2): Sequential(
      (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (batch_norm_2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (leaky_2): LeakyReLU(negative_slope=0.1, inplace=True)
    )
    (3): Sequential(
      (conv_3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (batch

In [83]:
read_dir = time.time()
batch_size=1
destination_path=""

In [84]:
orig_image=[]
image_data=[]
orig_image.append(stitched)
resized_img=prep_image(stitched,inp_dim)
image_data.append(resized_img)
#plt.imshow(cv2.cvtColor(orig_image[0],cv2.COLOR_RGB2BGR))

In [85]:
im_batches=image_data
im_dim_list=[(x.shape[1], x.shape[0]) for x in orig_image]
im_dim_list = torch.FloatTensor(im_dim_list).repeat(1,2)
if CUDA:
    im_dim_list = im_dim_list.cuda()

leftover = 0
if (len(im_dim_list) % batch_size):
    leftover = 1

leftover = 0
if (len(image_data) % batch_size):
    leftover = 1

if batch_size != 1:
    num_batches = len(image_data) // batch_size + leftover            
    im_batches = [torch.cat((im_batches[i*batch_size : min((i +  1)*batch_size,len(im_batches))]))  for i in range(num_batches)]  


In [86]:
write=0
start_det_loop = time.time()
for i, batch in enumerate(im_batches):
#load the image 
    start = time.time()
    if CUDA:
        batch = batch.cuda()
    with torch.no_grad():
        prediction = model(Variable(batch), CUDA)

    prediction = write_results(prediction, confidence, num_classes, nms_conf = nms_thesh)

    end = time.time()
    print("batch",i,"took time",(end-start))
    print("Each image took time = ",((end - start)/batch_size))
    if type(prediction) == int:
        print("No Objects Detected")
        continue

    prediction[:,0] += i*batch_size    

    if not write:                      #If we have't initialised output
        output = prediction  
        write = 1
    else:
        output = torch.cat((output,prediction))

    if CUDA:
        torch.cuda.synchronize()       

try:
    im_dim_list = torch.index_select(im_dim_list, 0, output[:,0].long())

    scaling_factor = torch.min(416/im_dim_list,1)[0].view(-1,1)


    output[:,[1,3]] -= (inp_dim - scaling_factor*im_dim_list[:,0].view(-1,1))/2
    output[:,[2,4]] -= (inp_dim - scaling_factor*im_dim_list[:,1].view(-1,1))/2

    output[:,1:5] /= scaling_factor



    for i in range(output.shape[0]):
        output[i, [1,3]] = torch.clamp(output[i, [1,3]], 0.0, im_dim_list[i,0])
        output[i, [2,4]] = torch.clamp(output[i, [2,4]], 0.0, im_dim_list[i,1])
except Exception as e:
    pass

batch 0 took time 2.758373498916626
Each image took time =  2.758373498916626


In [87]:
output_recast = time.time()
class_load = time.time()
colors = pkl.load(open(r"F:\Bublu\Project\pallete", "rb"))
draw = time.time()

def boxing(original_img, output):
    img = np.copy(original_img)
    '''
    for x in output:
        
        top_x = output[1]
        top_y = output[2]

        btm_x = output[3]
        btm_y = output[4]


        cls = int(output[-1])
        label = "{0}".format(classes[cls])


        newImage = cv2.rectangle(newImage, (top_x, top_y), (btm_x, btm_y), (255,0,0), 1)
        newImage = cv2.putText(newImage, label, (top_x, top_y-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.5, (0, 230, 0), 1, cv2.LINE_AA)
    '''
    for x in output:
        c1 = tuple(x[1:3].int())
        c2 = tuple(x[3:5].int())
        cls = int(x[-1])
        color = random.choice(colors)
        label = "{0}".format(classes[cls])
        if(label=="person"):
            cv2.rectangle(img, c1, c2, (255,0,0), 1)
            cv2.putText(img, label, (c1[0], c1[1]-5), cv2.FONT_HERSHEY_COMPLEX_SMALL , 0.5, (0, 230, 0), 1, cv2.LINE_AA)
    return img

In [88]:
j=0
k=0
for i,img in enumerate(orig_image):
    while(j<len(output) and output[j,0]==i):
        j+=1 
    newImage=boxing(orig_image[i],output[k:j,:])
    k=j

In [89]:
cv2.imshow("People Detected Image",newImage)
cv2.waitKey(0)

-1